# Exploratory Data Analysis

Исследовательский анализ данных для проекта предсказания отмены бронирований отелей

## Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Настройка отображения
plt.style.use('default')
sns.set_palette('husl')
%matplotlib inline

## Загрузка данных

Загружаем данные о бронированиях отелей

In [ ]:
# Загрузка данных (будет добавлено позже)
# df = pd.read_parquet('../data/hotel_bookings.parquet')
# print(f"Размер датасета: {df.shape}")
# df.head()

## Общая информация о данных

In [ ]:
# Основная информация о датасете
# df.info()
# print("\nПропущенные значения:")
# print(df.isnull().sum())

## Анализ целевой переменной

In [ ]:
# Распределение целевой переменной (отмены бронирований)
# plt.figure(figsize=(8, 6))
# df['is_canceled'].value_counts().plot(kind='bar')
# plt.title('Распределение отмен бронирований')
# plt.xlabel('Отменено (1) / Не отменено (0)')
# plt.ylabel('Количество')
# plt.show()